In [1]:
import pandas as pd
from googlesearch import search 
from urllib.request import urlretrieve
from urllib.request import Request, urlopen 
import requests  
import urllib.request

import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
import re
import tabula
import os
import warnings
warnings.filterwarnings("ignore")


from io import BytesIO

import re

In [2]:
csv_path = "/Users/Varun Sridharan/Desktop/EPICS/Xylem_Materials/epa_systems_with_sensus_groups_04-07-20.csv"

In [3]:
budget_df = pd.read_csv(csv_path,index_col=False)

In [4]:
budget_df["size_group"].value_counts()

< 500             406082
A: 500 - 2500      11491
B: 2500 - 7500      3825
C: > 7500           2599
Name: size_group, dtype: int64

In [5]:
filtered_budget = budget_df[((budget_df['size_group'] == 'A: 500 - 2500') | (budget_df['size_group'] == 'B: 2500 - 7500') | (budget_df['size_group'] == 'C: > 7500')) & (budget_df['owner_type_code'] == 'L')]
filtered_budget = filtered_budget.dropna(axis=0,how='all')
filtered_budget.shape

(14318, 52)

In [6]:
pws_names = filtered_budget['pws_name'][:].tolist()
pws_ids = filtered_budget['pwsid'][:].tolist()
categories = filtered_budget['size_group'][:].tolist()
pws_info = list(zip(pws_ids,pws_names,categories))

In [7]:
pws_info

[('083090091', 'TOWN OF BROWNING', 'A: 500 - 2500'),
 ('083890029', 'NEW TOWN| CITY OF', 'A: 500 - 2500'),
 ('084690020', 'ST. FRANCIS', 'A: 500 - 2500'),
 ('AK2110342', 'CITY AND BOROUGH OF JUNEAU', 'C: > 7500'),
 ('AK2110619', 'HAINES', 'A: 500 - 2500'),
 ('AK2111566', 'HAINES BOROUGH', 'A: 500 - 2500'),
 ('AK2120143', 'WRANGELL', 'A: 500 - 2500'),
 ('AK2120169', 'KLAWOCK', 'A: 500 - 2500'),
 ('AK2120232', 'KETCHIKAN PUBLIC UTILITIES', 'B: 2500 - 7500'),
 ('AK2130075', 'SITKA', 'B: 2500 - 7500'),
 ('AK2130148', 'PETERSBURG', 'A: 500 - 2500'),
 ('AK2130172', 'YAKUTAT PWS', 'A: 500 - 2500'),
 ('AK2210906', 'MOA MUNICIPALITY OF ANCHORAGE', 'C: > 7500'),
 ('AK2212021', 'GIRDWOOD WATER SYSTEM', 'A: 500 - 2500'),
 ('AK2224646', 'WASILLA WATER SYSTEM', 'A: 500 - 2500'),
 ('AK2226020', 'PALMER WATER SYSTEM', 'A: 500 - 2500'),
 ('AK2240448', 'KENAI WATER SYSTEM', 'A: 500 - 2500'),
 ('AK2240456', 'HOMER WATER SYSTEM', 'A: 500 - 2500'),
 ('AK2240757', 'SEWARD', 'A: 500 - 2500'),
 ('AK2241054', 

In [8]:

import re
def getWordsFromURL(url):
    return re.compile(r'[\:/?=\-&]+',re.UNICODE).split(url)


def check_keyword(keywords):
    for keyword in keywords:
        if ".pdf" in keyword:
            return True
    return False



def get_tables(url,req_data):
    class AppURLopener(urllib.request.FancyURLopener):
        version = "Mozilla/5.0"
        
        
    opener = AppURLopener()
    remoteFile = opener.open(url).read()
    memoryFile = BytesIO(remoteFile)
    
    print("Scanning url "+str(url))
    dfs = tabula.read_pdf(memoryFile, pages="all")
    req_data["total_number_of_tables"].append(len(dfs))
    pdfs = []
    for df in dfs:
        s = df.to_string()
        matches = ["water", "sewer", "capital","operational","utlities","p.u","distribution","meter"]
        if any(x in s.lower() for x in matches):
            pdfs.append(df)
            
    req_data["number_of_tables_with_budget_info"].append(len(pdfs))
    print("Found "+str(len(pdfs))+" tables")
    return req_data,pdfs




def main(pws_info):
    num_pdfs = 0
    urls_failed = []
    budget_tables_per_pdf = []
    missed = 0
    req_df = pd.DataFrame()
    req_data = {"pwsid":[],"pws_name":[],"category":[],"total_number_of_tables":[],"number_of_tables_with_budget_info":[],"missed_flag":[],"url_failed_to_download":[]}
    
    
    for pws_id,name,category in pws_info:
        print(pws_id,name,category)
        req_data["pwsid"].append(pws_id)
        req_data["pws_name"].append(name)
        req_data["category"].append(category)
        name = name.lower()
        query = name + " municipal utility water budget fiscal year 2020 pdf"
        urls = []
        print('-------------------------')
        print('Searching for query : '+str(query))
        print('--------------------------')
        for j in search(query, tld="com", num=10, stop=10, pause=2): 
            urls.append(j)
        for url in urls:
            words = getWordsFromURL(url)
            keywords = [x.lower() for x in words]
            if check_keyword(keywords):
                req_data["missed_flag"].append(0)
                num_pdfs += 1
                try:
                    req_data, budget_tables = get_tables(url,req_data)
                    budget_tables_per_pdf.append(budget_tables)
                    req_data["url_failed_to_download"].append(0)
                except:
                    req_data["url_failed_to_download"].append(1)
                    req_data["total_number_of_tables"].append(0)
                    req_data["number_of_tables_with_budget_info"].append(0)
                    print('URL failed to download - '+str(url))
                    urls_failed.append(url)
                    
            else:
                req_data["missed_flag"].append(1)
                req_data["total_number_of_tables"].append(0)
                req_data["number_of_tables_with_budget_info"].append(0)
                req_data["url_failed_to_download"].append(0)
                print('No query match')
                missed += 1

            break
        print(req_data)
    req_df = pd.DataFrame(req_data)
    return req_df,budget_tables_per_pdf

## Change this line to run

In [9]:
df,tables = main(pws_info[:10])

083090091 TOWN OF BROWNING A: 500 - 2500
-------------------------
Searching for query : town of browning municipal utility water budget fiscal year 2020 pdf
--------------------------
Scanning url https://www.chicago.gov/content/dam/city/depts/obm/supp_info/2020Budget/2020BudgetOverview.pdf


Got stderr: Aug 07, 2020 3:38:28 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:28 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:29 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:29 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:38:41 PM org.apache.pdfbox.pdmodel.font.PDType0Fo

Found 51 tables
{'pwsid': ['083090091'], 'pws_name': ['TOWN OF BROWNING'], 'category': ['A: 500 - 2500'], 'total_number_of_tables': [296], 'number_of_tables_with_budget_info': [51], 'missed_flag': [0], 'url_failed_to_download': [0]}
083890029 NEW TOWN| CITY OF A: 500 - 2500
-------------------------
Searching for query : new town| city of municipal utility water budget fiscal year 2020 pdf
--------------------------
No query match
{'pwsid': ['083090091', '083890029'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF'], 'category': ['A: 500 - 2500', 'A: 500 - 2500'], 'total_number_of_tables': [296, 0], 'number_of_tables_with_budget_info': [51, 0], 'missed_flag': [0, 1], 'url_failed_to_download': [0, 0]}
084690020 ST. FRANCIS A: 500 - 2500
-------------------------
Searching for query : st. francis municipal utility water budget fiscal year 2020 pdf
--------------------------
Scanning url https://www.sandiego.gov/sites/default/files/pb_full.pdf


Got stderr: Aug 07, 2020 3:39:25 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Aug 07, 2020 3:39:25 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Aug 07, 2020 3:39:33 PM org.apache

Found 365 tables
{'pwsid': ['083090091', '083890029', '084690020'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS'], 'category': ['A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500'], 'total_number_of_tables': [296, 0, 1597], 'number_of_tables_with_budget_info': [51, 0, 365], 'missed_flag': [0, 1, 0], 'url_failed_to_download': [0, 0, 0]}
AK2110342 CITY AND BOROUGH OF JUNEAU C: > 7500
-------------------------
Searching for query : city and borough of juneau municipal utility water budget fiscal year 2020 pdf
--------------------------
No query match
{'pwsid': ['083090091', '083890029', '084690020', 'AK2110342'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS', 'CITY AND BOROUGH OF JUNEAU'], 'category': ['A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'C: > 7500'], 'total_number_of_tables': [296, 0, 1597, 0], 'number_of_tables_with_budget_info': [51, 0, 365, 0], 'missed_flag': [0, 1, 0, 1], 'url_failed_to_download': [0, 0, 0, 0]}
AK2110619 HAINE

...

Found 26 tables
{'pwsid': ['083090091', '083890029', '084690020', 'AK2110342', 'AK2110619', 'AK2111566'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS', 'CITY AND BOROUGH OF JUNEAU', 'HAINES', 'HAINES BOROUGH'], 'category': ['A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'C: > 7500', 'A: 500 - 2500', 'A: 500 - 2500'], 'total_number_of_tables': [296, 0, 1597, 0, 0, 101], 'number_of_tables_with_budget_info': [51, 0, 365, 0, 0, 26], 'missed_flag': [0, 1, 0, 1, 1, 0], 'url_failed_to_download': [0, 0, 0, 0, 0, 0]}
AK2120143 WRANGELL A: 500 - 2500
-------------------------
Searching for query : wrangell municipal utility water budget fiscal year 2020 pdf
--------------------------
Scanning url https://www.wrangell.com/sites/default/files/fileattachments/finance/page/3372/fy_2020_budget_for_pdf_conversion_08.02.19.pdf


Got stderr: Aug 07, 2020 3:46:21 PM org.apache.pdfbox.rendering.TTFGlyph2D getPathForGID
Aug 07, 2020 3:46:22 PM org.apache.pdfbox.rendering.TTFGlyph2D getPathForGID
Aug 07, 2020 3:46:23 PM org.apache.pdfbox.rendering.TTFGlyph2D getPathForGID



Found 46 tables
{'pwsid': ['083090091', '083890029', '084690020', 'AK2110342', 'AK2110619', 'AK2111566', 'AK2120143'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS', 'CITY AND BOROUGH OF JUNEAU', 'HAINES', 'HAINES BOROUGH', 'WRANGELL'], 'category': ['A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'C: > 7500', 'A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500'], 'total_number_of_tables': [296, 0, 1597, 0, 0, 101, 78], 'number_of_tables_with_budget_info': [51, 0, 365, 0, 0, 26, 46], 'missed_flag': [0, 1, 0, 1, 1, 0, 0], 'url_failed_to_download': [0, 0, 0, 0, 0, 0, 0]}
AK2120169 KLAWOCK A: 500 - 2500
-------------------------
Searching for query : klawock municipal utility water budget fiscal year 2020 pdf
--------------------------
No query match
{'pwsid': ['083090091', '083890029', '084690020', 'AK2110342', 'AK2110619', 'AK2111566', 'AK2120143', 'AK2120169'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS', 'CITY AND BOROUGH OF JUNEAU', 'HAINE

Got stderr: Aug 07, 2020 3:47:24 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Aug 07, 2020 3:47:24 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Aug 07, 2020 3:47:24 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Aug 07, 2020 3:47:25 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Aug 07, 2020 3:47:26 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Aug 07, 2020 3:47:26 PM org.apache.pdfbox.conten

Found 64 tables
{'pwsid': ['083090091', '083890029', '084690020', 'AK2110342', 'AK2110619', 'AK2111566', 'AK2120143', 'AK2120169', 'AK2120232', 'AK2130075'], 'pws_name': ['TOWN OF BROWNING', 'NEW TOWN| CITY OF', 'ST. FRANCIS', 'CITY AND BOROUGH OF JUNEAU', 'HAINES', 'HAINES BOROUGH', 'WRANGELL', 'KLAWOCK', 'KETCHIKAN PUBLIC UTILITIES', 'SITKA'], 'category': ['A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'C: > 7500', 'A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'A: 500 - 2500', 'B: 2500 - 7500', 'B: 2500 - 7500'], 'total_number_of_tables': [296, 0, 1597, 0, 0, 101, 78, 0, 63, 164], 'number_of_tables_with_budget_info': [51, 0, 365, 0, 0, 26, 46, 0, 24, 64], 'missed_flag': [0, 1, 0, 1, 1, 0, 0, 1, 0, 0], 'url_failed_to_download': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [10]:
df

,pwsid,pws_name,category,total_number_of_tables,number_of_tables_with_budget_info,missed_flag,url_failed_to_download
0,083090091,TOWN OF BROWNING,A: 500 - 2500,296,51,0,0
1,083890029,NEW TOWN| CITY OF,A: 500 - 2500,0,0,1,0
2,084690020,ST. FRANCIS,A: 500 - 2500,1597,365,0,0
3,AK2110342,CITY AND BOROUGH OF JUNEAU,C: > 7500,0,0,1,0
4,AK2110619,HAINES,A: 500 - 2500,0,0,1,0
5,AK2111566,HAINES BOROUGH,A: 500 - 2500,101,26,0,0
6,AK2120143,WRANGELL,A: 500 - 2500,78,46,0,0
7,AK2120169,KLAWOCK,A: 500 - 2500,0,0,1,0
8,AK2120232,KETCHIKAN PUBLIC UTILITIES,B: 2500 - 7500,63,24,0,0
9,AK2130075,SITKA,B: 2500 - 7500,164,64,0,0
